# Language Models

In :numref:`sec_text-sequence`, we saw how to map text sequences into tokens, where these tokens can be viewed as a sequence of discrete observations such as words or characters. Assume that the tokens in a text sequence of length $T$ are in turn $x_1, x_2, \ldots, x_T$.
The goal of *language models*
is to estimate the joint probability of the whole sequence:

$$P(x_1, x_2, \ldots, x_T),$$

where statistical tools
in :numref:`sec_sequence`
can be applied.

Language models are incredibly useful. For instance, an ideal language model should generate natural text on its own, simply by drawing one token at a time $x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$.
Quite unlike the monkey using a typewriter, all text emerging from such a model would pass as natural language, e.g., English text. Furthermore, it would be sufficient for generating a meaningful dialog, simply by conditioning the text on previous dialog fragments.
Clearly we are still very far from designing such a system, since it would need to *understand* the text rather than just generate grammatically sensible content.

Nonetheless, language models are of great service even in their limited form.
For instance, the phrases "to recognize speech" and "to wreck a nice beach" sound very similar.
This can cause ambiguity in speech recognition,
which is easily resolved through a language model that rejects the second translation as outlandish.
Likewise, in a document summarization algorithm
it is worthwhile knowing that "dog bites man" is much more frequent than "man bites dog", or that "I want to eat grandma" is a rather disturbing statement, whereas "I want to eat, grandma" is much more benign.


In [1]:
import torch
from d2l import torch as d2l

## Learning Language Models

The obvious question is how we should model a document, or even a sequence of tokens. 
Suppose that we tokenize text data at the word level.
Let's start by applying basic probability rules:

$$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^T P(x_t  \mid  x_1, \ldots, x_{t-1}).$$

For example, 
the probability of a text sequence containing four words would be given as:

$$\begin{aligned}&P(\textrm{deep}, \textrm{learning}, \textrm{is}, \textrm{fun}) \\
=&P(\textrm{deep}) P(\textrm{learning}  \mid  \textrm{deep}) P(\textrm{is}  \mid  \textrm{deep}, \textrm{learning}) P(\textrm{fun}  \mid  \textrm{deep}, \textrm{learning}, \textrm{is}).\end{aligned}$$

### Markov Models and $n$-grams

Among those sequence model analyses in :numref:`sec_sequence`,
let's apply Markov models to language modeling.
A distribution over sequences satisfies the Markov property of first order if $P(x_{t+1} \mid x_t, \ldots, x_1) = P(x_{t+1} \mid x_t)$. Higher orders correspond to longer dependencies. This leads to a number of approximations that we could apply to model a sequence:

$$
\begin{aligned}
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2) P(x_3) P(x_4),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_2) P(x_4  \mid  x_3),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_1, x_2) P(x_4  \mid  x_2, x_3).
\end{aligned}
$$

The probability formulae that involve one, two, and three variables are typically referred to as *unigram*, *bigram*, and *trigram* models, respectively. 
In order to compute the language model, we need to calculate the
probability of words and the conditional probability of a word given
the previous few words.
Note that
such probabilities are
language model parameters.



### Word Frequency

Here, we
assume that the training dataset is a large text corpus, such as all
Wikipedia entries, [Project Gutenberg](https://en.wikipedia.org/wiki/Project_Gutenberg),
and all text posted on the
web.
The probability of words can be calculated from the relative word
frequency of a given word in the training dataset.
For example, the estimate $\hat{P}(\textrm{deep})$ can be calculated as the
probability of any sentence starting with the word "deep". A
slightly less accurate approach would be to count all occurrences of
the word "deep" and divide it by the total number of words in
the corpus.
This works fairly well, particularly for frequent
words. Moving on, we could attempt to estimate

$$\hat{P}(\textrm{learning} \mid \textrm{deep}) = \frac{n(\textrm{deep, learning})}{n(\textrm{deep})},$$

where $n(x)$ and $n(x, x')$ are the number of occurrences of singletons
and consecutive word pairs, respectively.
Unfortunately, 
estimating the
probability of a word pair is somewhat more difficult, since the
occurrences of "deep learning" are a lot less frequent. 
In particular, for some unusual word combinations it may be tricky to
find enough occurrences to get accurate estimates.
As suggested by the empirical results in :numref:`subsec_natural-lang-stat`,
things take a turn for the worse for three-word combinations and beyond.
There will be many plausible three-word combinations that we likely will not see in our dataset.
Unless we provide some solution to assign such word combinations a nonzero count, we will not be able to use them in a language model. If the dataset is small or if the words are very rare, we might not find even a single one of them.

### Laplace Smoothing

A common strategy is to perform some form of *Laplace smoothing*.
The solution is to
add a small constant to all counts. 
Denote by $n$ the total number of words in
the training set
and $m$ the number of unique words.
This solution helps with singletons, e.g., via

$$\begin{aligned}
	\hat{P}(x) & = \frac{n(x) + \epsilon_1/m}{n + \epsilon_1}, \\
	\hat{P}(x' \mid x) & = \frac{n(x, x') + \epsilon_2 \hat{P}(x')}{n(x) + \epsilon_2}, \\
	\hat{P}(x'' \mid x,x') & = \frac{n(x, x',x'') + \epsilon_3 \hat{P}(x'')}{n(x, x') + \epsilon_3}.
\end{aligned}$$

Here $\epsilon_1,\epsilon_2$, and $\epsilon_3$ are hyperparameters.
Take $\epsilon_1$ as an example:
when $\epsilon_1 = 0$, no smoothing is applied;
when $\epsilon_1$ approaches positive infinity,
$\hat{P}(x)$ approaches the uniform probability $1/m$. 
The above is a rather primitive variant of what
other techniques can accomplish :cite:`Wood.Gasthaus.Archambeau.ea.2011`.


Unfortunately, models like this get unwieldy rather quickly
for the following reasons. 
First, 
as discussed in :numref:`subsec_natural-lang-stat`,
many $n$-grams occur very rarely, 
making Laplace smoothing rather unsuitable for language modeling.
Second, we need to store all counts.
Third, this entirely ignores the meaning of the words. For
instance, "cat" and "feline" should occur in related contexts.
It is quite difficult to adjust such models to additional contexts,
whereas, deep learning based language models are well suited to
take this into account.
Last, long word
sequences are almost certain to be novel, hence a model that simply
counts the frequency of previously seen word sequences is bound to perform poorly there.
Therefore, we focus on using neural networks for language modeling
in the rest of the chapter.


## Perplexity

Next, let's discuss about how to measure the quality of the language model, which we will then use to evaluate our models in the subsequent sections.
One way is to check how surprising the text is.
A good language model is able to predict, with high accuracy, the tokens that come next.
Consider the following continuations of the phrase "It is raining", as proposed by different language models:

1. "It is raining outside"
1. "It is raining banana tree"
1. "It is raining piouw;kcj pwepoiut"

In terms of quality, Example 1 is clearly the best. The words are sensible and logically coherent.
While it might not quite accurately reflect which word follows semantically ("in San Francisco" and "in winter" would have been perfectly reasonable extensions), the model is able to capture which kind of word follows.
Example 2 is considerably worse by producing a nonsensical extension. Nonetheless, at least the model has learned how to spell words and some degree of correlation between words. Last, Example 3 indicates a poorly trained model that does not fit data properly.

We might measure the quality of the model by computing  the likelihood of the sequence.
Unfortunately this is a number that is hard to understand and difficult to compare.
After all, shorter sequences are much more likely to occur than the longer ones,
hence evaluating the model on Tolstoy's magnum opus
*War and Peace* will inevitably produce a much smaller likelihood than, say, on Saint-Exupery's novella *The Little Prince*. What is missing is the equivalent of an average.

Information theory comes handy here.
We defined entropy, surprisal, and cross-entropy
when we introduced the softmax regression
(:numref:`subsec_info_theory_basics`).
If we want to compress text, we can ask about
predicting the next token given the current set of tokens.
A better language model should allow us to predict the next token more accurately.
Thus, it should allow us to spend fewer bits in compressing the sequence.
So we can measure it by the cross-entropy loss averaged
over all the $n$ tokens of a sequence:

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$
:eqlabel:`eq_avg_ce_for_lm`

where $P$ is given by a language model and $x_t$ is the actual token observed at time step $t$ from the sequence.
This makes the performance on documents of different lengths comparable. For historical reasons, scientists in natural language processing prefer to use a quantity called *perplexity*. In a nutshell, it is the exponential of :eqref:`eq_avg_ce_for_lm`:

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

Perplexity can be best understood as the reciprocal of the geometric mean of the number of real choices that we have when deciding which token to pick next. Let's look at a number of cases:

* In the best case scenario, the model always perfectly estimates the probability of the target token as 1. In this case the perplexity of the model is 1.
* In the worst case scenario, the model always predicts the probability of the target token as 0. In this situation, the perplexity is positive infinity.
* At the baseline, the model predicts a uniform distribution over all the available tokens of the vocabulary. In this case, the perplexity equals the number of unique tokens of the vocabulary. In fact, if we were to store the sequence without any compression, this would be the best we could do for encoding it. Hence, this provides a nontrivial upper bound that any useful model must beat.

## Partitioning Sequences

We will design language models using neural networks
and use perplexity to evaluate 
how good the model is at 
predicting the next token given the current set of tokens
in text sequences.
Before introducing the model,
let's assume that it
processes a minibatch of sequences with predefined length
at a time.
Now the question is how to [**read minibatches of input sequences and target sequences at random**].


Suppose that the dataset takes the form of a sequence of $T$ token indices in `corpus`.
We will
partition it
into subsequences, where each subsequence has $n$ tokens (time steps).
To iterate over 
(almost) all the tokens of the entire dataset 
for each epoch
and obtain all possible length-$n$ subsequences,
we can introduce randomness.
More concretely,
at the beginning of each epoch,
discard the first $d$ tokens,
where $d\in [0,n)$ is uniformly sampled at random.
The rest of the sequence
is then partitioned
into $m=\lfloor (T-d)/n \rfloor$ subsequences.
Denote by $\mathbf x_t = [x_t, \ldots, x_{t+n-1}]$ the length-$n$ subsequence starting from token $x_t$ at time step $t$. 
The resulting $m$ partitioned subsequences
are 
$\mathbf x_d, \mathbf x_{d+n}, \ldots, \mathbf x_{d+n(m-1)}.$
Each subsequence will be used as an input sequence into the language model.


For language modeling,
the goal is to predict the next token based on the tokens we have seen so far; hence the targets (labels) are the original sequence, shifted by one token.
The target sequence for any input sequence $\mathbf x_t$
is $\mathbf x_{t+1}$ with length $n$.

![Obtaining five pairs of input sequences and target sequences from partitioned length-5 subsequences.](../img/lang-model-data.svg) 
:label:`fig_lang_model_data`

:numref:`fig_lang_model_data` shows an example of obtaining five pairs of input sequences and target sequences with $n=5$ and $d=2$.


In [2]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def __init__(self, batch_size, num_steps, num_train=10000, num_val=5000):
    super(d2l.TimeMachine, self).__init__()
    self.save_hyperparameters()
    corpus, self.vocab = self.build(self._download())
    array = torch.tensor([corpus[i:i+num_steps+1]
                        for i in range(len(corpus)-num_steps)])
    self.X, self.Y = array[:,:-1], array[:,1:]

To train language models,
we will randomly sample 
pairs of input sequences and target sequences
in minibatches.
The following data loader randomly generates a minibatch from the dataset each time.
The argument `batch_size` specifies the number of subsequence examples in each minibatch
and `num_steps` is the subsequence length in tokens.


In [3]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(
        self.num_train, self.num_train + self.num_val)
    return self.get_tensorloader([self.X, self.Y], train, idx)

As we can see in the following, 
a minibatch of target sequences
can be obtained 
by shifting the input sequences
by one token.


In [4]:
data = d2l.TimeMachine(batch_size=2, num_steps=10)
for X, Y in data.train_dataloader():
    print('X:', X, '\nY:', Y)
    break

X: tensor([[10,  4,  2, 21, 10, 16, 15,  0, 20,  2],
        [21,  9,  6, 19,  0, 24,  2, 26,  0, 16]]) 
Y: tensor([[ 4,  2, 21, 10, 16, 15,  0, 20,  2, 10],
        [ 9,  6, 19,  0, 24,  2, 26,  0, 16,  9]])


## Summary and Discussion

Language models estimate the joint probability of a text sequence. For long sequences, $n$-grams provide a convenient model by truncating the dependence. However, there is a lot of structure but not enough frequency to deal efficiently with infrequent word combinations via Laplace smoothing. Thus, we will focus on neural language modeling in subsequent sections.
To train language models, we can randomly sample pairs of input sequences and target sequences in minibatches. After training, we will use perplexity to measure the language model quality.

Language models can be scaled up with increased data size, model size, and amount in training compute. Large language models can perform desired tasks by predicting output text given input text instructions. As we will discuss later (e.g., :numref:`sec_large-pretraining-transformers`),
at the present moment
large language models form the basis of state-of-the-art systems across diverse tasks.


## Exercises


1. Suppose there are 100,000 words in the training dataset. How much word frequency and multi-word adjacent frequency does a four-gram need to store?

Total number of possible n-grams (there can be n-grams) in the word set $V$:

$$V^n$$

Where $V$ is the number of independent words in the vocabulary. So the total number of n-grams that actually appear (observed n-grams) in the data set $N$ (total number of words in dataset):

$$N - (n - 1)$$

Since each n-gram requires $n$ consecutive words, the actual number of n-grams is less $N$

Then **Four-gram frequencies (sequences of four adjacent words):**
- **Total four-grams in the dataset:** Each four-gram starts at a position in the text and consists of four consecutive words. The number of possible starting positions is:
$$
\text{Total four-grams} = \text{Total words} - 3 = 100,\!000 - 3 = 99,\!997
$$
- **Unique four-grams:** Since four-gram sequences may repeat, the number of unique four-grams is less than or equal to 99,997.
- **Frequency counts to store:** Up to 99,997 entries.

2. How would you model a dialogue?

A model that can be using to have a daily conversation with a human being is a very challenging task. The model needs to understand the context of the conversation and generate a response that is coherent with the context. The model should also be able to generate a response that is relevant to the conversation. There're three common approaches to model a dialogue:
- Rule-Based Systems: Use a set of rules and logic to generate responses. For example, the model uses a machine learning methods or regular expression (regex).
- Machine learning model: This system relies on deep learning or advanced language models to process conversations. For example:
    - (a) Retrieval-Based Models: The model searches for responses from a set of available responses based on the context or input sentence.
    - (b) Generative-Based Models:
        - RNN/LSTM/GRU: Good for processing time series, but struggles with long contexts.
        - Transformer (GPT, BERT, T5): Modern and efficient for natural language processing.
        - ChatGPT (GPT-based): Fine-tuned to generate natural conversational responses.

3. What other methods can you think of for reading long sequence data?

 Based on my research and some reading, I found that there are several methods to read long sequence data:

***Original Sentence (15 words)***: *"In the middle of the night, the stars shone brightly over the quiet town."*

 
1. **Truncation**: Truncation involves reducing a sequence to a fixed length by keeping only the first or last $n$ elements.
- This methods often a trade-off between computational feasibility and model performance. In contexts where early or late data points are more significant, choosing to keep the first or last elements makes sense. However, if important information is distributed throughout the sequence, truncation may severely impact the model's effectiveness.

- Example: Suppose we have a model only can process 10 words, the original sentence will be truncated to: *"In the middle of the night, the stars shone brightly"*

2. **Sliding Window**: A technique where a fixed-size window moves across the sequence, processing subsequences either with or without overlap.
- The sliding window approach balances context length and computational efficiency. Overlapping windows provide richer context but increase the number of computations. Non-overlapping windows are more efficient but may miss important transitional information.

- Example: If we apply a sliding window of 5 words with an overlap of 2 words.
    - Window 1: *"In the middle of the"*
    - Window 2: *"the middle of the night,"*
    - Window 3: *"middle of the night, the"*
    - And so on.

3. **Chunking (Partitioning)**: Dividing the sequence into smaller, fixed-size chunks and processing each independently.

- Example: Split the sentence into 3 equal chunks. We have:
    - *"In the middle of the"*
    - *"night, the stars shone"*
    - *"brightly over the quiet town."*

4. **Hierarchical Processing**: Processing data at multiple levels, starting with small parts and aggregating results to form a higher-level understanding.
- Hierarchical Processing is a method where data is processed at multiple levels of granularity, reflecting the inherent hierarchical structure present in many types of data, especially text. In language, this hierarchy typically follows the structure:
    - Characters form words.
    - Words form phrases.
    - Phrases form sentences.
    - Sentences form paragraphs.
    - Paragraphs form documents.

- Example: With our sentence, we could process the text at the word level, then phrase level, and finally sentence level:
    - Word Level: *"In", "the", "middle", "of", "the", "night,"...*
    - Phrase Level: *"In the middle of the night,"; "the stars shone brightly,"; "over the quiet town."*
    - Sentence Level: *"In the middle of the night, the stars shone brightly over the quiet town."*

5. **Attention Mechanisms**: Mechanisms that allow models to focus on specific parts of the input sequence when generating each part of the output.
- Self-attention calculates pairwise interactions between all elements, which is powerful but computationally intensive. Sparse attention mechanisms reduce the number of interactions, making it more efficient.

- Example: The methods will identify and focus on key words that contribute most to the sentence's meaning. The Attention Weights (hypothetical):
    - *"In"* - Low attention
    - *"the"* - Low attention
    - *"middle"* - Medium attention
    - *"of"* - Low attention
    - *"night,"* - High attention
    - *"stars"* - High attention
    - *"shone"* - High attention
    - *"brightly"* - High attention
    - *"over"* - Medium attention
    - *"quiet"* - High attention
    - *"town."* - Medium attention

6. **Transformer Variants for Long Sequences**: Modifications of the Transformer architecture designed to handle long sequences efficiently.
- These models address the quadratic scaling issue of self-attention in standard Transformers, enabling practical processing of long sequences.

4. Consider our method for discarding a uniformly random number of the first few tokens at the beginning of each epoch.
    1. Does it really lead to a perfectly uniform distribution over the sequences on the document?
    1. What would you have to do to make things even more uniform? 

I'm not sure how to answer this, but I think the method of discarding a uniformly random number of the first few tokens at the beginning of each epoch does not lead to a perfectly uniform distribution over the sequences on the document. The method is not perfect because it is random and the distribution of the tokens is not uniform. To make things even more uniform, we could try to shuffle the tokens in the document before discarding a uniformly random number of the first few tokens at the beginning of each epoch. This would help to ensure that the distribution of the tokens is more uniform and that the model is able to learn from a more diverse set of sequences.

5. If we want a sequence example to be a complete sentence, what kind of problem does this introduce in minibatch sampling? How can we fix it?

I don't have enough information to answer this one